### import libraries

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from math import pi
from shapely.geometry import Polygon, Point
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
%matplotlib inline

### import lightnings data

In [2]:
file = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/light/lightenings.csv.gz'

In [3]:
df_light = pd.read_csv(file, compression='gzip', sep=',', header=0, index_col=0)
#df_light

In [4]:
# create geometry points
gdf_light = gpd.GeoDataFrame(df_light, geometry=gpd.points_from_xy(df_light.CENTERLON, df_light.CENTERLAT), crs="EPSG:4269")
gdf_light

,#ZDAY,CENTERLON,CENTERLAT,TOTAL_COUNT,geometry
0,19920101,-97.8,33.6,1,POINT (-97.80000 33.60000)
1,19920101,-98.2,31.5,3,POINT (-98.20000 31.50000)
2,19920101,-99.0,36.2,1,POINT (-99.00000 36.20000)
3,19920101,-96.9,31.4,3,POINT (-96.90000 31.40000)
4,19920101,-80.4,24.6,1,POINT (-80.40000 24.60000)
...,...,...,...,...,...
70952947,20151231,-90.4,26.7,3,POINT (-90.40000 26.70000)
70952948,20151231,-86.9,29.3,3,POINT (-86.90000 29.30000)
70952949,20151231,-69.0,37.9,2,POINT (-69.00000 37.90000)
70952950,20151231,-89.7,28.1,3,POINT (-89.70000 28.10000)


### import extended forests shapes

In [5]:
ext_forest_file = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/temp/S_USA.ProclaimedForest/us_forests_ext_d100.shp'

gdf_forest_ext = gpd.read_file(ext_forest_file)
gdf_forest_ext

ERROR 1: PROJ: proj_create_from_database: Open of /cloud-home/XLOUIS/.magellan/conda/envs/geo/share/proj failed


,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
0,295435010328,Allegheny National Forest,740686.211,0.323984,3.965097,"MULTIPOLYGON (((-79.47447 41.54646, -79.47447 ..."
1,295380010328,Angeles National Forest,695598.432,0.276018,4.674530,"MULTIPOLYGON (((-118.39144 34.29308, -118.3923..."
2,295458010328,Angelina National Forest,398093.002,0.152499,3.691482,"MULTIPOLYGON (((-94.52605 31.25947, -94.52605 ..."
3,295372010328,Apache National Forest,1869703.247,0.736196,6.526785,"POLYGON ((-109.65438 34.13032, -109.65614 34.1..."
4,295400010328,Apalachicola National Forest,633591.300,0.240186,3.367179,"MULTIPOLYGON (((-85.10001 30.05484, -85.10021 ..."
...,...,...,...,...,...,...
149,295401010328,White River National Forest,2482446.167,1.052907,13.329421,"MULTIPOLYGON (((-108.12547 39.26733, -108.1254..."
150,109214010328,Whitman National Forest,1311082.638,0.605272,18.320346,"MULTIPOLYGON (((-117.72385 45.26576, -117.7286..."
151,295436010328,Willamette National Forest,1794216.105,0.816252,8.953350,"MULTIPOLYGON (((-122.62455 43.76543, -122.6247..."
152,295365010328,William B. Bankhead National Forest,349259.985,0.138292,2.451515,"MULTIPOLYGON (((-87.52237 34.42819, -87.52147 ..."


### join lightnings data and US forests

In [6]:
joined_gdf = gpd.sjoin(gdf_light, gdf_forest_ext[['PROCLAIMED', 'geometry']], how='left')
joined_gdf.loc[joined_gdf['PROCLAIMED'].notnull()].drop('index_right', axis=1)

,#ZDAY,CENTERLON,CENTERLAT,TOTAL_COUNT,geometry,PROCLAIMED
536,19920102,-95.3,30.7,1,POINT (-95.30000 30.70000),295455010328
597,19920103,-79.8,33.0,3,POINT (-79.80000 33.00000),295462010328
678,19920103,-79.8,33.1,9,POINT (-79.80000 33.10000),295462010328
840,19920103,-77.0,34.8,1,POINT (-77.00000 34.80000),295421010328
958,19920103,-76.8,34.8,1,POINT (-76.80000 34.80000),295421010328
...,...,...,...,...,...,...
70952339,20151231,-77.1,34.8,1,POINT (-77.10000 34.80000),295421010328
70952527,20151231,-76.9,34.9,2,POINT (-76.90000 34.90000),295421010328
70952529,20151231,-77.0,34.9,6,POINT (-77.00000 34.90000),295421010328
70952630,20151231,-85.6,32.5,2,POINT (-85.60000 32.50000),295368010328


In [15]:
df_grouped = joined_gdf.groupby(by=['PROCLAIMED', '#ZDAY']) \
                        .agg({'TOTAL_COUNT':'sum'}) \
                        .reset_index()
df_grouped = df_grouped.astype({'#ZDAY':'str'})
df_grouped['#ZDAY'] = pd.to_datetime(df_grouped['#ZDAY'], format='%Y%m%d')
df_grouped

,PROCLAIMED,#ZDAY,TOTAL_COUNT
0,105935010328,1992-02-23,3
1,105935010328,1992-03-06,4
2,105935010328,1992-03-07,2
3,105935010328,1992-03-08,5
4,105935010328,1992-03-17,1
...,...,...,...
254776,96813010328,2015-10-03,25
254777,96813010328,2015-10-04,9
254778,96813010328,2015-10-18,1
254779,96813010328,2015-10-19,6


### data analysis

In [9]:
print(df_grouped['TOTAL_COUNT'].sum(), 'impacts de foudre au total')
print(df_grouped['TOTAL_COUNT'].max(), 'impacts de foudre maxi')
print(df_grouped['TOTAL_COUNT'].min(), 'impacts de foudre mini')

30210694 impacts de foudre au total
17783 impacts de foudre maxi
1 impacts de foudre mini


In [10]:
# rows/days with more than 10'000 lightnings
df_grouped.loc[df_grouped['TOTAL_COUNT']>10000]

,PROCLAIMED,#ZDAY,TOTAL_COUNT
20420,295371010328,20060510,12511
20811,295371010328,20090506,10790
20844,295371010328,20090716,10888
21041,295371010328,20110425,17426
21291,295371010328,20130531,13491
21419,295371010328,20140709,17783
91125,295414010328,20030719,16932


In [29]:
# details of the day/forest with the day with the most lightning
joined_gdf.loc[(gdf_light['#ZDAY']==20140709) & (joined_gdf['PROCLAIMED']=='295371010328')].sort_values(by=['TOTAL_COUNT'], ascending=False)

,#ZDAY,CENTERLON,CENTERLAT,TOTAL_COUNT,geometry,index_right,PROCLAIMED
65491147,20140709,-94.3,34.7,623,POINT (-94.30000 34.70000),94.0,295371010328
65497255,20140709,-94.4,34.7,552,POINT (-94.40000 34.70000),94.0,295371010328
65498197,20140709,-93.4,34.8,424,POINT (-93.40000 34.80000),94.0,295371010328
65490000,20140709,-93.2,34.9,402,POINT (-93.20000 34.90000),94.0,295371010328
65489187,20140709,-93.3,34.8,400,POINT (-93.30000 34.80000),94.0,295371010328
...,...,...,...,...,...,...,...
65480085,20140709,-94.8,34.3,3,POINT (-94.80000 34.30000),94.0,295371010328
65484861,20140709,-94.7,34.2,3,POINT (-94.70000 34.20000),94.0,295371010328
65492351,20140709,-94.6,34.1,3,POINT (-94.60000 34.10000),94.0,295371010328
65487384,20140709,-94.6,33.9,2,POINT (-94.60000 33.90000),94.0,295371010328


In [30]:
# one forest without any lightning data
f = list(set(gdf_forest_ext['PROCLAIMED'].unique()) - set(df_grouped['PROCLAIMED'].unique()))
gdf_forest_ext.loc[gdf_forest_ext['PROCLAIMED']==f[0], 'FORESTNAME']

40    El Yunque National Forest
Name: FORESTNAME, dtype: object

In [31]:
# export weather data
out_path = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/light/light_data_forest.csv.gz'
df_grouped.to_csv(out_path, compression='gzip', index=False)

print('file_ok')

file_ok
